<a href="https://colab.research.google.com/github/terrysu64/Machine-Learning-and-Data-Science-Projects/blob/main/introduction_to_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#NLP Fundamentals

In [1]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-08-21 16:23:44--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-08-21 16:23:45 (98.6 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [2]:
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, walk_through_dir

### import dataset
kaggle: https://www.kaggle.com/competitions/nlp-getting-started

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2022-08-21 16:23:47--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.4.128, 74.125.24.128, 142.251.10.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.4.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2022-08-21 16:23:47 (132 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [4]:
unzip_data('nlp_getting_started.zip')

### visualize text dataset

In [5]:
import pandas as pd
train_df = pd.read_csv('train.csv').sample(frac=1,
                                           random_state=42)
test_df = pd.read_csv('test.csv').sample(frac=1,
                                         random_state=42)
train_df.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [6]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [7]:
len(train_df), len(test_df)

(7613, 3263)

In [8]:
from random import randint
for _ in range(3):
  i = randint(1,7612)
  print(f"{'disaster' if train_df.target[i] else 'not disaster'}: {train_df.text[i]}")

disaster: when a monster truck racer catches on fire at the fair
disaster: EPA begins demolition of homes in toxic area #Buffalo - http://t.co/noRkXBRS6G
not disaster: Some drugs and alcohol in Jackson Vroman house.

http://t.co/5OQhQ8QUQV


### split data

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train_df.text.to_numpy(),
                                                  train_df.target.to_numpy(),
                                                  test_size=0.1,
                                                  random_state=42)

In [10]:
train_df.head().to_numpy() #changes df => numpy arr

array([[3796, 'destruction', nan,
        'So you have a new weapon that can cause un-imaginable destruction.',
        1],
       [3185, 'deluge', nan,
        'The f$&amp;@ing things I do for #GISHWHES Just got soaked in a deluge going for pads and tampons. Thx @mishacollins @/@',
        0],
       [7769, 'police', 'UK',
        'DT @georgegalloway: RT @Galloway4Mayor: \x89ÛÏThe CoL police can catch a pickpocket in Liverpool Stree... http://t.co/vXIn1gOq4Q',
        1],
       [191, 'aftershock', nan,
        'Aftershock back to school kick off was great. I want to thank everyone for making it possible. What a great night.',
        0],
       [9810, 'trauma', 'Montgomery County, MD',
        'in response to trauma Children of Addicts develop a defensive self - one that decreases vulnerability. (3',
        0]], dtype=object)

### converting text => numbers

In [11]:
#tokenization: word => a specific number
#embedding: representing relationships between tokens (can limit size + can be learned)

import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

MAX_VOCAB=10000
MAX_LENGTH=15
vectorizer = TextVectorization(max_tokens=MAX_VOCAB, #limit to how many words in vocab
                               standardize="lower_and_strip_punctuation",
                               split="whitespace",
                               ngrams=None, #create grops of n words
                               output_mode="int",
                               output_sequence_length=MAX_LENGTH, #how long to limit tweet
                               pad_to_max_tokens=True #replace anything not in max_tokens to 0
)

In [12]:
vectorizer.adapt(X_train)

In [13]:
vectorizer(['hi there buddy'])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[1663,   74,    1,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0]])>

In [14]:
vocab = vectorizer.get_vocabulary()
vocab[:5] #top 5 words

['', '[UNK]', 'the', 'a', 'in']

### creating embedding layer

In [15]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=MAX_VOCAB,
                             output_dim=128,
                             input_length=MAX_LENGTH) #neural networks work well with %8==0

embedding(vectorizer("hi there buddy"))

<tf.Tensor: shape=(15, 128), dtype=float32, numpy=
array([[ 0.04545389,  0.01759062, -0.02088803, ..., -0.02447306,
        -0.01918215, -0.01868059],
       [ 0.01769186, -0.03195822, -0.04973723, ..., -0.04155694,
        -0.01736398, -0.03528029],
       [ 0.04123275,  0.00529229,  0.04244262, ...,  0.01114079,
        -0.00459039,  0.01399114],
       ...,
       [-0.04662377, -0.00387279,  0.01117498, ...,  0.02733809,
         0.00478716,  0.02736313],
       [-0.04662377, -0.00387279,  0.01117498, ...,  0.02733809,
         0.00478716,  0.02736313],
       [-0.04662377, -0.00387279,  0.01117498, ...,  0.02733809,
         0.00478716,  0.02736313]], dtype=float32)>

## building models for the dataset
###multinomial naive bayes using TF-IDF formula

note: it is common to use no DL algos first

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", MultinomialNB())
])

model.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [17]:
baseline_score = model.score(X_val, y_val)
baseline_score

0.7926509186351706

In [18]:
model.predict(X_val)[:5], y_val[:5]

(array([1, 1, 1, 0, 0]), array([0, 0, 1, 1, 1]))

In [19]:
from sklearn.metrics import classification_report

report = classification_report(model.predict(X_val),
                      y_val)
report

'              precision    recall  f1-score   support\n\n           0       0.93      0.75      0.83       516\n           1       0.63      0.89      0.73       246\n\n    accuracy                           0.79       762\n   macro avg       0.78      0.82      0.78       762\nweighted avg       0.83      0.79      0.80       762\n'

In [20]:
#               precision    recall  f1-score   support

#            0       0.93      0.75      0.83       516
#            1       0.63      0.89      0.73       246

#     accuracy                           0.79       762
#    macro avg       0.78      0.82      0.78       762
# weighted avg       0.83      0.79      0.80       762


model 1: RNN (LSTM)
connections between tensors form a directed or undirected graph along a temporal sequence.

In [21]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(64, return_sequences=True)(x)
x = layers.LSTM(64, return_sequences=False)(x)
print(x.shape)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = tf.keras.Model(inputs,outputs,name="model_LSTM")

(None, 64)


In [22]:
model.summary()

Model: "model_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 15, 64)            49408     
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                        

In [23]:
model.compile(loss="binary_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"]) 

In [24]:
history = model.fit(X_train, 
                    y_train,
                    epochs=5,
                    validation_data=(X_val, y_val))

Epoch 1/5
215/215 [==============================] - 18s 15ms/step - loss: 0.5078 - accuracy: 0.7549 - val_loss: 0.4536 - val_accuracy: 0.7913
Epoch 2/5
215/215 [==============================] - 2s 8ms/step - loss: 0.3179 - accuracy: 0.8691 - val_loss: 0.5021 - val_accuracy: 0.7769
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.2184 - accuracy: 0.9190 - val_loss: 0.5098 - val_accuracy: 0.7835
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1587 - accuracy: 0.9437 - val_loss: 0.8376 - val_accuracy: 0.7585
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1150 - accuracy: 0.9574 - val_loss: 0.8458 - val_accuracy: 0.7598


### model 2: GRU

In [25]:
inputs = layers.Input(shape=(1,), dtype="string")
x = vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64, return_sequences=True)(x)
x = layers.LSTM(64, return_sequences=False)(x)
print(x.shape)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = tf.keras.Model(inputs,outputs,name="model_GRU")

(None, 64)


In [26]:
model.compile(loss="binary_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"]) 

In [27]:
history = model.fit(X_train, 
                    y_train,
                    epochs=5,
                    validation_data=(X_val, y_val))

Epoch 1/5
215/215 [==============================] - 5s 11ms/step - loss: 0.1904 - accuracy: 0.9238 - val_loss: 0.8705 - val_accuracy: 0.7717
Epoch 2/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0880 - accuracy: 0.9634 - val_loss: 1.0213 - val_accuracy: 0.7638
Epoch 3/5
215/215 [==============================] - 2s 7ms/step - loss: 0.0766 - accuracy: 0.9682 - val_loss: 1.0545 - val_accuracy: 0.7743
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0658 - accuracy: 0.9710 - val_loss: 1.2261 - val_accuracy: 0.7756
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0571 - accuracy: 0.9740 - val_loss: 1.2450 - val_accuracy: 0.7690


###model 3: Bidirectional-LSTM
creates representation for text (l to r and vice-versa)

In [31]:
inputs = layers.Input(shape=(1,), dtype="string")
x = vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.GRU(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=False))(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = tf.keras.Model(inputs,outputs,name="model_LSTM_bidirectional")

In [32]:
model.summary()

Model: "model_LSTM_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional_4 (Bidirectio  (None, 15, 128)          74496     
 nal)                                                            
                                                                 
 bidirectional_5 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                          

In [33]:
model.compile(loss="binary_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"]) 

In [34]:
history = model.fit(X_train, 
                    y_train,
                    epochs=5,
                    validation_data=(X_val, y_val))

Epoch 1/5
215/215 [==============================] - 11s 17ms/step - loss: 0.1353 - accuracy: 0.9507 - val_loss: 0.9160 - val_accuracy: 0.7520
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0666 - accuracy: 0.9731 - val_loss: 1.4944 - val_accuracy: 0.7467
Epoch 3/5
215/215 [==============================] - 2s 12ms/step - loss: 0.0479 - accuracy: 0.9771 - val_loss: 1.3606 - val_accuracy: 0.7690
Epoch 4/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0490 - accuracy: 0.9772 - val_loss: 1.3528 - val_accuracy: 0.7598
Epoch 5/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0511 - accuracy: 0.9766 - val_loss: 1.3235 - val_accuracy: 0.7703
